# Preliminary Notebook – PyTorch

## Goal: Determine whether an SEM image is usable or not

This notebook sets up the initial framework for classifying SEM images as 'usable' or 'not usable' based on image quality.

**Assumptions**:
- Dataset is labeled with `usable` / `not_usable` categories.
- Images are grayscale SEM images stored in folders `data/processed/usable/` and `data/processed/not_usable/`.

In [ ]:
# Install dependencies if necessary
# !pip install torch torchvision matplotlib

In [ ]:
import os
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define transforms
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# Load dataset
data_dir = 'data/processed/'
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

class_names = dataset.classes
print(f"Classes: {class_names}")

In [ ]:
# Use a pretrained model (e.g., ResNet18)
model = models.resnet18(pretrained=True)
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 2)  # 2 classes: usable, not usable
model = model.to(device)

In [ ]:
# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
# Dummy training loop (1 epoch)
model.train()
for images, labels in dataloader:
    images, labels = images.to(device), labels.to(device)
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print("Preliminary training loop finished.")